In [ ]:
import os
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
os.environ['KERAS_BACKEND'] = 'tensorflow'
import numpy as np
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.optimizers import SGD

In [ ]:
import resnet_sd
import freezing

In [2]:
# Per channel mean and std normalization
def color_preprocessing(x_train, x_val, x_test):
    
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')    
    x_test = x_test.astype('float32')
    
    mean = np.mean(x_train, axis=(0,1,2))  # Per channel mean
    std = np.std(x_train, axis=(0,1,2))
    x_train = (x_train - mean) / std
    x_val = (x_val - mean) / std
    x_test = (x_test - mean) / std
    
    return x_train, x_val, x_test

In [ ]:
img_rows, img_cols = 32, 32
img_channels = 3
nb_epochs = 500
batch_size = 128
nb_classes = 100
seed = 333


# data
(x_train, y_train), (x_test, y_test) = cifar100.load_data()


# Data splitting (get additional 5k validation set)
# Sklearn to split
x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=seed)  # random_state = seed
x_train45, x_val, x_test = color_preprocessing(x_train45, x_val, x_test)


img_gen = ImageDataGenerator(
    horizontal_flip=True,
    data_format="channels_last",
    width_shift_range=0.125,  # 0.125*32 = 4 so max padding of 4 pixels, as described in paper.
    height_shift_range=0.125,
    fill_mode="constant",
    cval = 0
)

img_gen.fit(x_train45)

y_train45 = np_utils.to_categorical(y_train45, nb_classes)  # 1-hot vector
y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)


# building and training net
model = resnet_sd.resnet_sd_model(img_shape = (32,32), img_channels = 3,
                        layers = 110, nb_classes = nb_classes, verbose = True)

sgd = SGD(learning_rate=0.1, decay=1e-4, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint('model_resnet110SD_c100_best.hdf5', verbose=1, save_best_only=True)

In [4]:
freezing_list = []
for i in range(len(model.layers)):
  if i < len(model.layers) * 0.8:
    freezing_list.append(int(nb_epochs*0.6))
freezing_list.append(nb_epochs)

In [5]:
freezing.training_with_freezing(model, img_gen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list, batch_size=128,lr_schedule = [[0, 0.1],[nb_epochs*0.5,0.01],[nb_epochs*0.75,0.001]],cbks=[checkpointer], name='resnet_sd_cifar100')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_31
.........vars
......activation_32
.........vars
......activa

2023-04-20 11:13:15.963916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200


351/351 [==============================] - ETA: 0s - loss: 4.7726 - accuracy: 0.0984
Epoch 1: val_loss improved from inf to 4.91106, saving model to model_resnet110SD_c100_best.hdf5
351/351 [==============================] - 62s 122ms/step - loss: 4.7726 - accuracy: 0.0984 - val_loss: 4.9111 - val_accuracy: 0.1148 - lr: 0.1000
Epoch 2/300
351/351 [==============================] - ETA: 0s - loss: 4.1542 - accuracy: 0.1769
Epoch 2: val_loss improved from 4.91106 to 4.00222, saving model to model_resnet110SD_c100_best.hdf5
351/351 [==============================] - 37s 106ms/step - loss: 4.1542 - accuracy: 0.1769 - val_loss: 4.0022 - val_accuracy: 0.2108 - lr: 0.1000
Epoch 3/300
351/351 [==============================] - ETA: 0s - loss: 3.8330 - accuracy: 0.2261
Epoch 3: val_loss improved from 4.00222 to 3.91068, saving model to model_resnet110SD_c100_best.hdf5
351/351 [==============================] - 37s 104ms/step - loss: 3.8330 - accuracy: 0.2261 - val_loss: 3.9107 - val_accuracy: 0

[24.89,
 0.13913255594223736,
 0.2953859572101412,
 1.2317225282402715,
 0.22313591422790868]